# aT 농산품 예측 base line

&nbsp;

## 모듈 가져오기

In [84]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 3

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

## 입력 shape 및 형태 정의 함수

In [85]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [86]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [87]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [88]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [89]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.002, verbose = 1):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [90]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [91]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [92]:
for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1419/1419 [00:00<00:00, 71299.40it/s]


&nbsp;

## Test 과정

In [ ]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)

        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists('./model_output') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 38ms/step


2022-09-01 16:16:13.814997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 16:16:13.930954: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 35ms/step


2022-09-01 16:16:14.044224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 16:16:14.150810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-01 16:17:15.871929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 74ms/step


2022-09-01 16:17:16.081153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 61ms/step


2022-09-01 16:17:16.314714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-01 16:17:16.478442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [01:03<00:00,  6.30s/it]


&nbsp;

## 정답 제출 파일생성

In [ ]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  5.23it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.2800,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,4010.2180,1274.486800,4152.300300,1602.558800,345.014280,3175.380400,2513.2542,3674.744100,3490.030800,1045.201900,...,8719.250000,304.951780,6.278003,48.830406,2763.178000,3061.719200,2579.786000,1670.883800,1841.851600,1273.903700
2,3425.8008,663.017500,3946.863000,1366.238300,354.067230,3197.351300,2527.6853,3596.737800,2966.023200,1031.955800,...,6377.046400,288.547940,7.225252,216.359480,2115.840600,3155.327100,2805.118000,1593.719500,1653.569500,1297.211700
3,1331.3796,475.995450,1069.459000,1288.427100,351.256230,3189.434800,2506.0576,1424.195600,208.273500,987.363340,...,2879.781000,201.790880,6.438143,165.246890,98.739220,3135.564200,-79.765434,1638.713000,448.106780,1293.540000
4,3996.8232,1244.307500,5087.919000,1693.385500,347.120670,3146.936800,2491.6057,4171.572000,3245.354500,1051.638100,...,9300.624000,206.518550,8.039433,193.506800,2418.688200,3630.155500,3311.626200,1680.267700,2004.952500,1302.802200
5,4103.5600,1277.105500,4318.174300,1904.789400,350.541930,3162.900100,2505.7910,4177.147000,3414.382000,1063.103600,...,8598.991000,255.892430,6.029510,227.746520,2331.620000,2916.526900,3164.286600,1693.429700,1997.534900,1327.684600
6,3939.3508,1300.134200,4594.152000,1870.413700,355.472530,3158.036400,2476.8948,4387.514000,3531.918500,1055.991700,...,8438.967000,294.780150,7.364776,182.000820,2278.482700,2858.214400,2591.370600,1690.235200,1722.246000,1318.172100
7,3753.7178,1280.308500,4465.989700,1951.158200,351.315120,3160.953900,2463.8374,3825.167700,3267.746000,1076.719100,...,8115.673000,304.400940,7.334653,251.109360,2426.748500,2897.916500,2864.647700,1649.009900,1709.534300,1317.167100
8,3993.1938,1247.143100,4155.241700,1423.454300,348.965330,3190.738000,2464.0770,3343.472200,2878.704800,1036.814500,...,7379.530000,305.344500,8.665063,21.993930,2313.196500,3051.437300,2371.627000,1592.720700,1712.574100,1273.168600
9,3435.8042,686.618650,3648.192100,1147.715700,359.032870,3208.490500,2484.1208,2956.795000,2901.518600,1129.259200,...,4784.182000,289.936460,8.662633,235.883620,1969.290600,3242.777800,2776.775600,1707.372200,1644.170400,1297.356600


- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,0.035931,-0.064334,0.427013,-0.528670,-0.912606,-0.656862,-0.075085,0.093338,-0.289474,-0.108964,...,-0.994321,-0.968824,-0.239520,-0.438702,-0.540897,-0.680557,-0.365994,-0.389798,0,d+1
1,-0.115037,-0.513245,0.356411,-0.598174,-0.910313,-0.654488,-0.069774,0.070129,-0.396156,-0.120256,...,-0.993464,-0.861864,-0.417680,-0.421541,-0.500797,-0.695310,-0.430805,-0.378633,0,d+2
2,-0.656074,-0.650547,-0.632461,-0.621059,-0.911025,-0.655343,-0.077733,-0.576262,-0.957598,-0.158271,...,-0.994176,-0.894497,-0.972825,-0.425164,-1.014195,-0.686708,-0.845752,-0.380392,0,d+3
3,0.032471,-0.086490,0.748556,-0.501957,-0.912073,-0.659936,-0.083052,0.241158,-0.339287,-0.103477,...,-0.992727,-0.876454,-0.334330,-0.334492,-0.410658,-0.678763,-0.309851,-0.375955,0,d+4
4,0.060043,-0.062412,0.484019,-0.439780,-0.911206,-0.658211,-0.077831,0.242817,-0.304875,-0.093703,...,-0.994545,-0.854593,-0.358293,-0.465320,-0.436878,-0.676247,-0.312404,-0.364037,0,d+5
5,0.017624,-0.045505,0.578864,-0.449891,-0.909957,-0.658736,-0.088465,0.305407,-0.280947,-0.099766,...,-0.993338,-0.883800,-0.372917,-0.476010,-0.538835,-0.676858,-0.407165,-0.368593,0,d+6
6,-0.030329,-0.060060,0.534818,-0.426143,-0.911010,-0.658421,-0.093271,0.138094,-0.334729,-0.082095,...,-0.993365,-0.839677,-0.332112,-0.468731,-0.490203,-0.684739,-0.411540,-0.369075,0,d+7
7,0.031533,-0.084409,0.428024,-0.581346,-0.911605,-0.655202,-0.093183,-0.005224,-0.413933,-0.116114,...,-0.992161,-0.985958,-0.363363,-0.440587,-0.577941,-0.695501,-0.410494,-0.390150,0,d+8
8,-0.112453,-0.495918,0.253767,-0.662444,-0.909055,-0.653284,-0.085806,-0.120271,-0.409288,-0.037305,...,-0.992163,-0.849398,-0.458013,-0.405509,-0.505840,-0.673581,-0.434040,-0.378564,0,d+9
9,-0.636858,-0.639287,-0.634499,-0.651308,-0.909224,-0.653832,-0.081967,-0.464527,-0.925071,-0.167743,...,-0.991999,-0.875329,-0.889311,-0.413185,-0.994733,-0.686921,-0.904197,-0.379905,0,d+10


- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [ ]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.035931,-0.064334,0.427013,-0.528670,-0.912606,-0.656862,-0.075085,0.093338,...,0.009078,-0.493440,-0.994321,-0.968824,-0.239520,-0.438702,-0.540897,-0.680557,-0.365994,-0.389798
1,0,d+2,-0.115037,-0.513245,0.356411,-0.598174,-0.910313,-0.654488,-0.069774,0.070129,...,-0.261985,-0.520689,-0.993464,-0.861864,-0.417680,-0.421541,-0.500797,-0.695310,-0.430805,-0.378633
2,0,d+3,-0.656074,-0.650547,-0.632461,-0.621059,-0.911025,-0.655343,-0.077733,-0.576262,...,-0.666723,-0.664802,-0.994176,-0.894497,-0.972825,-0.425164,-1.014195,-0.686708,-0.845752,-0.380392
3,0,d+4,0.032471,-0.086490,0.748556,-0.501957,-0.912073,-0.659936,-0.083052,0.241158,...,0.076360,-0.656949,-0.992727,-0.876454,-0.334330,-0.334492,-0.410658,-0.678763,-0.309851,-0.375955
4,0,d+5,0.060043,-0.062412,0.484019,-0.439780,-0.911206,-0.658211,-0.077831,0.242817,...,-0.004840,-0.574934,-0.994545,-0.854593,-0.358293,-0.465320,-0.436878,-0.676247,-0.312404,-0.364037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.333259,-0.150941,0.215028,-0.123271,-0.654930,-0.766785,-0.822911,NaN,...,-0.049888,-0.714168,-0.993062,-1.523844,-0.015295,-0.186742,-0.617940,0.037620,-0.085884,-0.493653
146,9,d+12,-0.345852,-0.168063,0.130232,-0.162403,-0.678327,-0.770218,-0.816687,NaN,...,-0.070860,-0.557075,-0.993295,0.396585,-0.026242,-0.304954,-0.587855,0.128025,-0.089726,-0.484652
147,9,d+13,-0.402864,-0.127313,0.165152,-0.161243,-0.618772,-0.766023,-0.829573,NaN,...,-0.062880,-0.562561,-0.994497,-0.268160,-0.096355,-0.382257,-0.636426,-0.046316,-0.166886,-0.491094
148,9,d+14,-0.411495,-0.119988,0.148485,-0.131628,-0.668598,-0.765657,-0.824809,NaN,...,-0.064678,-0.551531,-0.994093,-0.453503,-0.047449,-0.335234,-0.663255,0.001727,-0.139577,-0.490446


- 제출 양식

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.035931,-0.064334,0.427013,-0.528670,-0.912606,-0.656862,-0.075085,0.093338,...,0.009078,-0.493440,-0.994321,-0.968824,-0.239520,-0.438702,-0.540897,-0.680557,-0.365994,-0.389798
1,0,d+2,-0.115037,-0.513245,0.356411,-0.598174,-0.910313,-0.654488,NaN,0.070129,...,-0.261985,-0.520689,-0.993464,-0.861864,-0.417680,-0.421541,-0.500797,-0.695310,-0.430805,-0.378633
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.077733,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.032471,-0.086490,0.748556,-0.501957,-0.912073,-0.659936,NaN,0.241158,...,0.076360,-0.656949,-0.992727,-0.876454,-0.334330,-0.334492,-0.410658,-0.678763,-0.309851,-0.375955
4,0,d+5,0.060043,-0.062412,0.484019,-0.439780,-0.911206,-0.658211,NaN,0.242817,...,-0.004840,-0.574934,-0.994545,-0.854593,-0.358293,-0.465320,-0.436878,-0.676247,-0.312404,-0.364037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.333259,-0.150941,0.215028,-0.123271,-0.654930,-0.766785,-0.822911,NaN,...,-0.049888,-0.714168,-0.993062,-1.523844,-0.015295,-0.186742,-0.617940,0.037620,-0.085884,-0.493653
146,9,d+12,-0.345852,-0.168063,0.130232,-0.162403,-0.678327,-0.770218,-0.816687,NaN,...,-0.070860,-0.557075,-0.993295,0.396585,-0.026242,-0.304954,-0.587855,0.128025,-0.089726,-0.484652
147,9,d+13,-0.402864,-0.127313,0.165152,-0.161243,-0.618772,-0.766023,NaN,NaN,...,-0.062880,-0.562561,-0.994497,-0.268160,-0.096355,-0.382257,-0.636426,-0.046316,-0.166886,-0.491094
148,9,d+14,-0.411495,-0.119988,0.148485,-0.131628,-0.668598,-0.765657,-0.824809,NaN,...,-0.064678,-0.551531,-0.994093,-0.453503,-0.047449,-0.335234,-0.663255,0.001727,-0.139577,-0.490446
